# Test setup
Test if everything installed right

In [ ]:
import detection_test
import detection_infer_speed 
import mxnext

> ⚠️ **Attention**:
- for `operator_cxx` run `make` in current dir (adding support for `cython`)
- missing mxnext: `pipenv run pip3 install 'git+https://github.com/RogerChern/mxnext#egg=mxnext'`

# Playing around with the shell
## Datasets
Download `COCO` dataset and plot `#images`

In [ ]:
# !./coco_data.sh # download COCO dataset
!ls -l ./data/coco/images/* | wc -l # amount of images

### Prepare custom dataset for `mahdabschaltung`

1. Setup dir structure
2. Split dataset in `training`/`test` (80/20)
3. Create `roidb` for MXNet (converting `VOC` format to `COCO` and saving in data/cache/

In [ ]:
""" 1. Setup dir structure
data
└── src
    └── mahd
        ├── Annotations
        ├── ImageSets
        │   └── Main
        │       ├── test.txt
        │       └── train.txt
        └── JPEGImages
"""
!rm -r data/src/mahd/*

!mkdir -p data/src/mahd/Annotations
!mkdir -p data/src/mahd/ImageSets/Main
!mkdir -p data/src/mahd/JPEGImages

# Copy Annotations
!ln -s $HOME/Documents/Bachelor/mahdabschaltung/dev/data/annotations_voc/tractor_vedai/* data/src/mahd/Annotations/
!ln -s $HOME/Documents/Bachelor/mahdabschaltung/dev/data/annotations_voc/tractor_google/* data/src/mahd/Annotations/
!ln -s $HOME/Documents/Bachelor/mahdabschaltung/dev/data/annotations_voc/schwader_google/* data/src/mahd/Annotations/

# Copy Images
!ln -s $HOME/Documents/Bachelor/mahdabschaltung/dev/data/images/tractor_vedai/* data/src/mahd/JPEGImages/
!ln -s $HOME/Documents/Bachelor/mahdabschaltung/dev/data/images/tractor_google/* data/src/mahd/JPEGImages/
!ln -s $HOME/Documents/Bachelor/mahdabschaltung/dev/data/images/schwader_google/* data/src/mahd/JPEGImages/

In [ ]:
# 2. Split dataset in `training`/`test` (80/20)
!rm data/src/mahd/ImageSets/Main/*
!./utils/create_image_split_sets.sh

!ls data/src/mahd/ImageSets/Main/

In [ ]:
# 3. Create `roidb` for MXNet (converting `VOC` format to `COCO` and saving in data/cache/
!python3 utils/create_voc_roidb.py --data-dir data/src/mahd --split train
!python3 utils/create_voc_roidb.py --data-dir data/src/mahd --split test

!ls data/cache # check if mahd_test.roidb & mahd_train.roidb created

## Prepare config `(config/*.py)`
- set `batch_size` to 1 (OOM otherwise for *consumer* GPUs

``` diff
@@ -12,7 +12,7 @@ def get_config(is_train):
     class General:
         log_frequency = 10
         name = __name__.rsplit("/")[-1].rsplit(".")[-1]
-        batch_image = 2 if is_train else 1
+        batch_image = 1 if is_train else 1
         fp16 = False
         loader_worker
```

- update GPU: `[0]` meaning only one GPU in use

``` diff
@@ -20,7 +20,7 @@ def get_config(is_train):
     class KvstoreParam:
         kvstore     = "nccl"
         batch_image = General.batch_image
-        gpus        = [0, 1, 2, 3, 4, 5, 6, 7]
+        gpus        = [0]
         fp16        = General.fp16
```

- update classes to match actual classes (`index out of range errors otherwise [cid]`)

``` diff
@@ -80,7 +80,7 @@ def get_config(is_train):
             bg_thr_lo = 0.0
 
         class bbox_target:
-            num_reg_class = 81
+            num_reg_class = 1 + 3
             class_agnostic = False
             weight = (1.0, 1.0, 1.0, 1.0)
             mean = (0.0, 0.0, 0.0, 0.0)
@@ -90,7 +90,7 @@ def get_config(is_train):
     class BboxParam:
         fp16 = General.fp16
         normalizer = NormalizeParam.normalizer
-        num_class   = 1 + 80
+        num_class   = 1 + 3
         image_roi   = 512
         batch_image = General.batch_image
```

- point to actual imageset

``` diff
@@ -111,9 +111,9 @@ def get_config(is_train):
 
     class DatasetParam:
         if is_train:
-            image_set = ("coco_train2017", )
+            image_set = ("mahd_train", )
         else:
-            image_set = ("coco_val2017", )
+            image_set = ("mahd_test", 
```

- it's not the coco dataset!

``` diff
@@ -190,6 +190,7 @@ def get_config(is_train):
 
         class coco:
             annotation = "data/coco/annotations/instances_minival2014.json"
+            annotation = None
```

- add tensorboard support

``` diff
@@ -269,27 +270,34 @@ def get_config(is_train):
         label_name = []
 
     import core.detection_metric as metric
+    from mxboard import SummaryWriter
+
+    sw = SummaryWriter(logdir="./tflogs", flush_secs=5)  
 
     rpn_acc_metric = metric.AccWithIgnore(
         "RpnAcc",
         ["rpn_cls_loss_output", "rpn_cls_label_blockgrad_output"],
-        []
+        label_names=[],
+        summary=sw
     )
     rpn_l1_metric = metric.L1(
         "RpnL1",
         ["rpn_reg_loss_output", "rpn_cls_label_blockgrad_output"],
-        []
+        label_names=[],
+        summary=sw
     )
     # for bbox, the label is generated in network so it is an output
     box_acc_metric = metric.AccWithIgnore(
         "RcnnAcc",
         ["bbox_cls_loss_output", "bbox_label_blockgrad_output"],
-        []
+        label_names=[],
+        summary=sw
     )
     box_l1_metric = metric.L1(
         "RcnnL1",
         ["bbox_reg_loss_output", "bbox_label_blockgrad_output"],
-        []
+        label_names=[],
+        summary=sw
     )
 
     metric_list = [rpn_acc_metric, rpn_l1_metric, box_acc_metric, box_l1_metric]
```

## Download pretrained models

- https://github.com/TuSimple/simpledet/tree/master/models/tridentnet
- https://github.com/TuSimple/simpledet/blob/master/MODEL_ZOO.md

## `Train`, `test` & check `inference speed`

In [ ]:
!python3 detection_train.py --config config/faster_r50v1_fpn_1x.py

10-05 02:04:43 parameter shape
10-05 02:04:43 [('data', (1, 3, 800, 1333)),
 ('conv0_weight', (64, 3, 7, 7)),
 ('bn0_gamma', (64,)),
 ('bn0_beta', (64,)),
 ('bn0_moving_mean', (64,)),
 ('bn0_moving_var', (64,)),
 ('stage1_unit1_conv1_weight', (64, 64, 1, 1)),
 ('stage1_unit1_bn1_gamma', (64,)),
 ('stage1_unit1_bn1_beta', (64,)),
 ('stage1_unit1_bn1_moving_mean', (64,)),
 ('stage1_unit1_bn1_moving_var', (64,)),
 ('stage1_unit1_conv2_weight', (64, 64, 3, 3)),
 ('stage1_unit1_bn2_gamma', (64,)),
 ('stage1_unit1_bn2_beta', (64,)),
 ('stage1_unit1_bn2_moving_mean', (64,)),
 ('stage1_unit1_bn2_moving_var', (64,)),
 ('stage1_unit1_conv3_weight', (256, 64, 1, 1)),
 ('stage1_unit1_bn3_gamma', (256,)),
 ('stage1_unit1_bn3_beta', (256,)),
 ('stage1_unit1_bn3_moving_mean', (256,)),
 ('stage1_unit1_bn3_moving_var', (256,)),
 ('stage1_unit1_sc_weight', (256, 64, 1, 1)),
 ('stage1_unit1_sc_bn_gamma', (256,)),
 ('stage1_unit1_sc_bn_beta', (256,)),
 ('stage1_unit1_sc_bn_moving_mean', (256,)),
 ('stage1

In [ ]:
!python3 detection_test.py --config config/faster_r50v1_fpn_1x.py

In [ ]:
!python3 detection_infer_speed.py --config config/faster_r50v1_fpn_1x.py --shape 1000 800

### Inference on single image and plot

In [ ]:
!python3 infer.py